This code is for the Mid Term project for winter class **DSC540** for Data Science Master in Bellebue University, NE.
- Anirban Pal

*Please refer to the related paper **Process Details.pdf** for detail about the data and consequent challenges.*

### Import data

In [1]:
# Import required modules and set up parameters
import pandas as pd
import datetime as dt
import pandas_profiling as pp
from fuzzywuzzy import fuzz

maindata = 'Data\googleplaystore.csv'
reviewdata = 'Data\googleplaystore_user_reviews.csv'

c:\users\anirban\appdata\local\programs\python\python36-32\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Read google play store app details and examine
appinfo = pd.read_csv(maindata) 
appinfo.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"""i DT"" Fútbol. Todos Somos Técnicos.",SPORTS,NaN,27,3.6M,500+,Free,0,Everyone,Sports,7-Oct-17,0.22,4.1 and up
1,.R,TOOLS,4.5,259,203k,"10,000+",Free,0,Everyone,Tools,16-Sep-14,1.1.06,1.5 and up
2,/u/app,COMMUNICATION,4.7,573,53M,"10,000+",Free,0,Mature 17+,Communication,3-Jul-18,4.2.4,4.1 and up
3,[adult swim],FAMILY,3.6,21433,21M,"1,000,000+",Free,0,Mature 17+,Entertainment,29-May-18,3.0.1805181047,5.0 and up
4,[BN] Blitz,SPORTS,3.2,4,5.6M,100+,Free,0,Everyone,Sports,17-May-18,4.6,4.3 and up


In [3]:
# Raad user review and examine
reviewinfo = pd.read_csv(reviewdata)
reviewinfo.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


### Replace header

I am happy with the headers in the `reviewinfo` data frame. But in the `appinfo` dataframe, we have a few columns with spaces and I would like to replace those headers with same text but "\_" as a separator instead of space.

In [4]:
appheader = list(appinfo)
appheadernew = [hdr.replace(' ', '_') for hdr in appheader]

# Replace header and examine
appinfo.columns = appheadernew

print(appinfo.columns)

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Current_Ver',
       'Android_Ver'],
      dtype='object')


### Identify Outliers and Bad Data + Format data

Before the transformation steps, it is a good idea to profile and examine the data to decide the next course of action.

In [5]:
pp.ProfileReport(appinfo)

Number of variables,13
Number of observations,10841
Total Missing (%),1.1%
Total size in memory,592.9 KiB
Average record size in memory,56.0 B
Numeric,1
Categorical,12
Boolean,0
Date,0
Text (Unique),0
Rejected,0


First of all, there is a visible formatting requirement in the Last Update date. I plan to convert it from string to mm-dd-yyyy format.

In [6]:
# Change the string dd-mmm-yy to date mm-dd-yyyy format and examine
appinfo.Last_Updated = pd.to_datetime(appinfo.Last_Updated, errors='coerce')
appinfo.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Current_Ver,Android_Ver
0,"""i DT"" Fútbol. Todos Somos Técnicos.",SPORTS,NaN,27,3.6M,500+,Free,0,Everyone,Sports,2017-10-07,0.22,4.1 and up
1,.R,TOOLS,4.5,259,203k,"10,000+",Free,0,Everyone,Tools,2014-09-16,1.1.06,1.5 and up
2,/u/app,COMMUNICATION,4.7,573,53M,"10,000+",Free,0,Mature 17+,Communication,2018-07-03,4.2.4,4.1 and up
3,[adult swim],FAMILY,3.6,21433,21M,"1,000,000+",Free,0,Mature 17+,Entertainment,2018-05-29,3.0.1805181047,5.0 and up
4,[BN] Blitz,SPORTS,3.2,4,5.6M,100+,Free,0,Everyone,Sports,2018-05-17,4.6,4.3 and up


Before doing further formatting, I would like to see the unique values of some of the columns of my interest.

In [7]:
# Category column
appinfo.Category.unique()

array(['SPORTS', 'TOOLS', 'COMMUNICATION', 'FAMILY', 'PERSONALIZATION',
       'SOCIAL', 'NEWS_AND_MAGAZINES', 'HEALTH_AND_FITNESS',
       'BOOKS_AND_REFERENCE', 'LIFESTYLE', 'BUSINESS', 'SHOPPING',
       'MEDICAL', 'GAME', 'FINANCE', 'COMICS', 'PHOTOGRAPHY',
       'TRAVEL_AND_LOCAL', 'DATING', 'PRODUCTIVITY', 'ART_AND_DESIGN',
       'FOOD_AND_DRINK', 'VIDEO_PLAYERS', 'HOUSE_AND_HOME',
       'MAPS_AND_NAVIGATION', 'ENTERTAINMENT', 'EVENTS', 'EDUCATION',
       'WEATHER', 'AUTO_AND_VEHICLES', 'BEAUTY', 'PARENTING',
       'LIBRARIES_AND_DEMO', '1.9'], dtype=object)

As we can see, among all other legitimate categories, we have `1.9` as a category. We would like to change this values to `OTHER`.

In [8]:
appinfo.Category = appinfo.Category.replace('1.9','OTHER')

### Find the duplicate rows in the data frame

In [62]:
appinfo[appinfo.duplicated(keep=False)]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Current_Ver,Android_Ver
16,10 Best Foods for You,HEALTH_AND_FITNESS,4.0,2490,3.8M,"500,000+",Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up
17,10 Best Foods for You,HEALTH_AND_FITNESS,4.0,2490,3.8M,"500,000+",Free,0,Everyone 10+,Health & Fitness,2017-02-17,1.9,2.3.3 and up
30,1800 Contacts - Lens Store,MEDICAL,4.7,23160,26M,"1,000,000+",Free,0,Everyone,Medical,2018-07-27,7.4.1,5.0 and up
31,1800 Contacts - Lens Store,MEDICAL,4.7,23160,26M,"1,000,000+",Free,0,Everyone,Medical,2018-07-27,7.4.1,5.0 and up
48,21-Day Meditation Experience,HEALTH_AND_FITNESS,4.4,11506,15M,"100,000+",Free,0,Everyone,Health & Fitness,2018-08-02,3.0.0,4.1 and up
49,21-Day Meditation Experience,HEALTH_AND_FITNESS,4.4,11506,15M,"100,000+",Free,0,Everyone,Health & Fitness,2018-08-02,3.0.0,4.1 and up
91,420 BZ Budeze Delivery,MEDICAL,5.0,2,11M,100+,Free,0,Mature 17+,Medical,2018-06-06,1.0.1,4.1 and up
92,420 BZ Budeze Delivery,MEDICAL,5.0,2,11M,100+,Free,0,Mature 17+,Medical,2018-06-06,1.0.1,4.1 and up
124,8fit Workouts & Meal Planner,HEALTH_AND_FITNESS,4.6,115721,67M,"10,000,000+",Free,0,Everyone,Health & Fitness,2018-07-11,3.5.0,5.0 and up
125,8fit Workouts & Meal Planner,HEALTH_AND_FITNESS,4.6,115721,67M,"10,000,000+",Free,0,Everyone,Health & Fitness,2018-07-11,3.5.0,5.0 and up


In [63]:
# Number of rows in the data frame before removing duplicates
len(appinfo)

10841

In [64]:
# Remove the duplicate rows
appinfo = appinfo.drop_duplicates(keep=False)

In [65]:
# Number of rows in the data frame after removing duplicates
len(appinfo)

9953

### Fuzzy matching of "Category" column to lump the apps of same category

In [14]:
fuzz.partial_ratio(appinfo[0])

100

In [19]:
appinfo.index[1]

1